<a href="https://colab.research.google.com/github/adhwikaguptha/readability-analysis-with-nlp-techniques/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl requests beautifulsoup4 nltk


In [ ]:
import pandas as pd

# Load the Excel file


# Load the Excel file
input_file = "/content/sample_data/Input.xlsx"  # Change this to your actual file path
df = pd.read_excel(input_file)

# Display the first few rows to check the structure
df.head()

df = pd.read_excel(input_file)

# Display the first few rows to check the structure
df.head()


In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)
        soup = BeautifulSoup(response.text, "html.parser")


        for script in soup(["script", "style"]):  # Remove scripts and styles
            script.decompose()

        text = " ".join(soup.stripped_strings)  # Join text while stripping extra spaces
        return text
    except Exception as e:
        print(f"Failed to extract text from {url}: {e}")
        return ""


df['Extracted Text'] = df['URL'].apply(extract_text_from_url)
df.head()


in the above code
BeautifulSoup extracts HTML elements from the webpage.
soup.stripped_strings extracts all visible text elements.
" ".join(...) ensures proper spacing between words.


In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)


df['Cleaned Text'] = df['Extracted Text'].apply(clean_text)

# Display results
df.head()


In the above code Iam cleaning the code by converting it into lower case .Cleaning it by removing stop words and special characters nltk. Ensures that the stopword dataset is available.
Retrieves a list of common English
Converts it into a set for faster lookup.

In [ ]:
df.to_excel("preprocessed_output.xlsx", index=False)
print("Preprocessed data saved to preprocessed_output.xlsx")


In [ ]:
from google.colab import files
files.download("preprocessed_output.xlsx")


In [ ]:
import importlib.util

def is_installed(package_name):
    return importlib.util.find_spec(package_name) is not None
import importlib.util

print("textstat installed:", is_installed("textstat"))

In [ ]:
import sys
import subprocess

def install_if_missing(package_name):
    if not is_installed(package_name):
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

install_if_missing("pandas")
install_if_missing("openpyxl")
install_if_missing("nltk")
install_if_missing("textstat")

print("All required packages are installed!")


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from textstat import textstat

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


In [ ]:
def load_word_list(filepath):
    """ Load words from a file into a set """
    with open(filepath, "r", encoding="latin-1") as file:
        return {line.strip().lower() for line in file if line.strip()}

positive_words = load_word_list("/content/sample_data/positive-words.txt")
negative_words = load_word_list("/content/sample_data/negative-words.txt")
def get_positive_score(text):
    words = text.split()
    return sum(1 for word in words if word.lower() in positive_words)

def get_negative_score(text):
    words = text.split()
    return sum(1 for word in words if word.lower() in negative_words) * -1  # Convert to positive value

def get_polarity_score(pos_score, neg_score):
    return (pos_score - neg_score) / ((pos_score + neg_score) + 0.000001)

def get_subjectivity_score(pos_score, neg_score, total_words):
    return (pos_score + neg_score) / (total_words + 0.000001)
def get_avg_sentence_length(text):
    sentences = nltk.sent_tokenize(text)
    num_sentences = len(sentences)
    num_words = len(text.split())
    return num_words / num_sentences if num_sentences > 0 else 0

def get_complex_word_count(text):
    words = text.split()
    return sum(1 for word in words if syllable_count(word) > 2)

def get_percentage_complex_words(complex_word_count, total_words):
    return (complex_word_count / total_words) * 100 if total_words > 0 else 0

def get_fog_index(avg_sentence_length, percentage_complex_words):
    return 0.4 * (avg_sentence_length + percentage_complex_words)

def get_avg_words_per_sentence(text):
    return get_avg_sentence_length(text)

def get_word_count(text):
    words = text.split()
    return len([word for word in words if word not in stop_words])

def syllable_count(word):
    """ Count syllables in a word """
    word = word.lower()
    count = len(re.findall(r'[aeiouy]+', word))
    count -= word.endswith(("es", "ed"))
    return max(1, count)

def get_syllables_per_word(text):
    words = text.split()
    total_syllables = sum(syllable_count(word) for word in words)
    return total_syllables / len(words) if len(words) > 0 else 0

def get_personal_pronouns(text):
    """ Count occurrences of personal pronouns (excluding 'US') """
    pronouns = re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE)
    return len(pronouns)

def get_avg_word_length(text):
    words = text.split()
    total_chars = sum(len(word) for word in words)
    return total_chars / len(words) if len(words) > 0 else 0
df["Positive Score"] = df["Cleaned Text"].apply(get_positive_score)
df["Negative Score"] = df["Cleaned Text"].apply(get_negative_score)
df["Polarity Score"] = df.apply(lambda x: get_polarity_score(x["Positive Score"], x["Negative Score"]), axis=1)
df["Subjectivity Score"] = df.apply(lambda x: get_subjectivity_score(x["Positive Score"], x["Negative Score"], len(x["Cleaned Text"].split())), axis=1)

df["Average Sentence Length"] = df["Cleaned Text"].apply(get_avg_sentence_length)
df["Complex Word Count"] = df["Cleaned Text"].apply(get_complex_word_count)
df["Percentage of Complex Words"] = df.apply(lambda x: get_percentage_complex_words(x["Complex Word Count"], len(x["Cleaned Text"].split())), axis=1)
df["Fog Index"] = df.apply(lambda x: get_fog_index(x["Average Sentence Length"], x["Percentage of Complex Words"]), axis=1)
df["Average Number of Words Per Sentence"] = df["Cleaned Text"].apply(get_avg_words_per_sentence)
df["Word Count"] = df["Cleaned Text"].apply(get_word_count)
df["Syllables Per Word"] = df["Cleaned Text"].apply(get_syllables_per_word)
df["Personal Pronouns"] = df["Cleaned Text"].apply(get_personal_pronouns)
df["Average Word Length"] = df["Cleaned Text"].apply(get_avg_word_length)

# Saving the results
df.to_excel("sentiment_analysis_output.xlsx", index=False)
print("Sentiment analysis completed! Results saved to sentiment_analysis_output.xlsx")


In [ ]:
import nltk

nltk.download('punkt_tab')

In [ ]:
import pandas as pd
import re
import nltk

nltk.download('punkt')


In [ ]:
def syllable_count(word):
    """ Count syllables in a word """
    word = word.lower()
    count = len(re.findall(r'[aeiouy]+', word))
    count -= word.endswith(("es", "ed"))
    return max(1, count)

def get_syllables_per_word(text):
    words = text.split()
    total_syllables = sum(syllable_count(word) for word in words)
    return total_syllables / len(words) if len(words) > 0 else 0
def get_personal_pronouns(text):
    """ Count occurrences of personal pronouns (excluding 'US') """
    pronouns = re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE)
    return len(pronouns)
def get_avg_word_length(text):
    words = text.split()
    total_chars = sum(len(word) for word in words)
    return total_chars / len(words) if len(words) > 0 else 0
def get_complex_word_count(text):
    words = text.split()
    return sum(1 for word in words if syllable_count(word) > 2)

def get_percentage_complex_words(complex_word_count, total_words):
    return (complex_word_count / total_words) * 100 if total_words > 0 else 0
def get_avg_sentence_length(text):
    sentences = nltk.sent_tokenize(text)
    num_sentences = len(sentences)
    num_words = len(text.split())
    return num_words / num_sentences if num_sentences > 0 else 0

def get_fog_index(avg_sentence_length, percentage_complex_words):
    return 0.4 * (avg_sentence_length + percentage_complex_words)

df = pd.read_excel("preprocessed_output.xlsx")

df["Syllables Per Word"] = df["Cleaned Text"].apply(get_syllables_per_word)
df["Personal Pronouns"] = df["Cleaned Text"].apply(get_personal_pronouns)
df["Average Word Length"] = df["Cleaned Text"].apply(get_avg_word_length)

df["Complex Word Count"] = df["Cleaned Text"].apply(get_complex_word_count)
df["Percentage of Complex Words"] = df.apply(lambda x: get_percentage_complex_words(x["Complex Word Count"], len(x["Cleaned Text"].split())), axis=1)

df["Average Sentence Length"] = df["Cleaned Text"].apply(get_avg_sentence_length)
df["Fog Index"] = df.apply(lambda x: get_fog_index(x["Average Sentence Length"], x["Percentage of Complex Words"]), axis=1)


df.to_excel("advanced_text_analysis.xlsx", index=False)
print("Advanced text analysis completed! Results saved to advanced_text_analysis.xlsx")


In [ ]:
from google.colab import files
files.download("sentiment_analysis_output.xlsx")


In [ ]:

generated_df = pd.read_excel("advanced_text_analysis.xlsx")  # Your computed results
expected_df = pd.read_excel("/content/sample_data/Output Data Structure.xlsx")  # Expected results provided by the company

# Display first few rows
print("Generated Data Preview:")
display(generated_df.head())

print("\nExpected Data Preview:")
display(expected_df.head())


In [ ]:
# Fill missing text values with an empty string
generated_df["Cleaned Text"].fillna("", inplace=True)


In [ ]:
# If URL is empty, flag it
generated_df["Missing URL"] = generated_df["URL"].isna()


In [ ]:
# Ensure text encoding consistency
generated_df["Cleaned Text"] = generated_df["Cleaned Text"].astype(str).apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))


In [ ]:
# Save the verified and cleaned results
generated_df.to_excel("final_output.xlsx", index=False)
print("Final output saved as final_output.xlsx")


In [ ]:
from google.colab import files
files.download("final_output.xlsx")